<h2>Linear Regression Submission 01</h2>

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
train = pd.read_csv(r'D:\Resume Projects\Backpack Prediction Challenge\train.csv')
test = pd.read_csv(r'D:\Resume Projects\Backpack Prediction Challenge\test.csv')

In [3]:
binary_cols = ['Laptop Compartment', 'Waterproof']
for col in binary_cols:
    mode_val = train[col].mode()[0]
    train[col].fillna(mode_val, inplace=True)
    test[col].fillna(mode_val, inplace=True)
    train[col] = train[col].map({'Yes': 1, 'No': 0}).astype(int)
    test[col] = test[col].map({'Yes': 1, 'No': 0}).astype(int)

C:\Users\omjad\AppData\Local\Temp\ipykernel_8788\3352363872.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna(mode_val, inplace=True)
C:\Users\omjad\AppData\Local\Temp\ipykernel_8788\3352363872.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe

In [4]:
categorical_cols = ['Brand', 'Material', 'Size', 'Style', 'Color']
for col in categorical_cols:
    train[col] = train[col].str.strip().str.lower()
    test[col] = test[col].str.strip().str.lower()

In [5]:
X_train = train.drop(['id', 'Price'], axis=1)
y_train = train['Price']
X_test = test.drop('id', axis=1)

In [6]:
numerical_cols = ['Compartments', 'Weight Capacity (kg)'] + binary_cols
categorical_cols = ['Brand', 'Material', 'Size', 'Style', 'Color']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numerical_cols),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_cols)
    ])

In [7]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Compartments',
                                                   'Weight Capacity (kg)',
                                                   'Laptop Compartment',
                                                   'Waterproof']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Brand', 'Material', 'Size',
                                                   'Style', 'Color'])])),
                ('regressor', LinearRegression())])

In [8]:
predictions = model.predict(X_test)
submission = pd.DataFrame({
    'id': test['id'],
    'Price': predictions.round(3)
})
submission.to_csv('LR_submission.csv', index=False)